Importing all neccesary libraries

In [ ]:
import numpy as np
import os
import tensorflow as tf
import cv2
from google.colab.patches import cv2_imshow
from skimage import color
from skimage import io
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import shutil

Mounting google drive to google colab

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Generating data with attributes like rescaling, zooming etc

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.4,
    zoom_range=0.4,
    horizontal_flip=True,
    )

test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    "/content/drive/MyDrive/pneumonia/chest_xray/train",
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    color_mode = "rgb"
    )

In [ ]:
test_generator = test_datagen.flow_from_directory(
    "/content/drive/MyDrive/pneumonia/chest_xray/test",
    target_size=(150, 150),
    batch_size=32,
    shuffle = False,
    class_mode='categorical',
    color_mode = "rgb"
    )

In [ ]:
validation_generator = test_datagen.flow_from_directory(
    "/content/drive/MyDrive/pneumonia/chest_xray/val",
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    color_mode = "rgb"
    )

Defining the model using CNN.

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(150, 150,3)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(3, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Defining validation functions

In [ ]:
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.callbacks import ReduceLROnPlateau
earlyStopping = EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='min')
mcp_save = ModelCheckpoint('.mdl_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, epsilon=1e-4, mode='min')

Training the model

In [ ]:
history = model.fit(
        train_generator,
        steps_per_epoch=38,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=7,
        callbacks=[mcp_save, reduce_lr_loss,earlyStopping])

In [ ]:
probabilities = model.predict_generator(test_generator)

Ploting graph of loss and accuracy

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

Checking accuracy on test data

In [ ]:
test_loss, test_acc = model.evaluate(test_generator)
print ('Test loss: {}, Test accuracy: {}'.format(test_loss, test_acc*100))